In [1]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Creamos el dataframes de las corrientes calientes

In [2]:
Hot_datos = {0: [0,0], 1: [0,0], 2: [50,0], 3: [25,0],4: [125,190], 5:[0,0]}
Hot_df = pd.DataFrame(Hot_datos,index=['H1','H2'])
Hot_df

,0,1,2,3,4,5
H1,0,0,50,25,125,0
H2,0,0,0,0,190,0


E igualmente de las frías.

In [3]:
Cold_datos =  {0: [0,0], 1:[60,0], 2: [40,0], 3: [20,20], 4: [100,100], 5:[20,20]}
Cold_df = pd.DataFrame(Cold_datos,index=['C1','C2'])
Cold_df

,0,1,2,3,4,5
C1,0,60,40,20,100,20
C2,0,0,0,20,100,20


In [4]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.k=Set(initialize=Hot_df.columns)

Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [5]:
model.pprint()

3 Set Declarations
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'H1', 'H2'}
    j : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    k : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

3 Declarations: i j k


Creamos ahora las variables del modelo. Para cada tipo de utility creamos una variable para cada intervalo, luego impondremos en qué intervalos esa utility es cero.

In [6]:
Qhps = model.Qhps = Var(model.k,within = NonNegativeReals)
Qlps = model.Qlps = Var(model.k,within = NonNegativeReals)
Qw = model.Qw = Var(model.k,within = NonNegativeReals)
R = model.R = Var(model.k,within = NonNegativeReals)

Creamos la función de coste a partir de los consumos de utilities y sus precios. Aunque las unidades no coincidan, para la solución da igual. Es decir los precios están en $/kWyr y los consumos en MW. Tendría que coger como base de cálculo un año y los MW transformarlos en kWyr. Como cada término se multiplicaría por el mismo valor, no hay problema porque la solución sería la misma (aunque no sería igual el valor de la FO).

In [7]:
model.coste = Objective(expr = Qhps[1]*80+Qlps[3]*50 + Qw[5]*20)

Constraints

In [8]:
ni = list(model.k)[1:] 
model.int = ConstraintList()
for i in ni:
    model.int.add(
        R[i-1]+Qhps[i]+Qlps[i]+sum(Hot_df[i]) == R[i]+sum(Cold_df[i])+Qw[i]
    )

model.R0 = Constraint(expr = R[0] == 0) 
model.R9 = Constraint(expr = R[5] == 0)

nii = list(model.k)[0:1]+list(model.k)[2:] 
model.hps = ConstraintList()
for i in nii:
    model.hps.add(
        Qhps[i]==0
    )

niii = list(model.k)[0:3]+list(model.k)[4:] 
model.lps = ConstraintList()
for i in niii:
    model.lps.add(
        Qlps[i]==0
    )

niiii = list(model.k)[0:5] 
model.cw = ConstraintList()
for i in niiii:
    model.cw.add(
        Qw[i]==0
    )

Resolvemos el modelo

In [9]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

7 Set Declarations
    cw_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    hps_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'H1', 'H2'}
    int_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    j : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    k : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    lps_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dime

In [10]:
Qh = value(model.Qhps[1])
Ql = value(model.Qlps[3])
Qw = value(model.Qw[5])
print('Cold utility = {0:2.2f}, LP Steam = {1:2.2f}, HP Steam = {2:2.2f}'.format(Qw, Ql, Qh))

Cold utility = 75.00, LP Steam = 5.00, HP Steam = 60.00


In [11]:
del model
model = ConcreteModel()

In [12]:
s=hotst.copy()
s.insert(0,'LPS')
s.insert(1,'HPS')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize=['LPS','HPS'])
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)

intmax=max(Hot_df.columns)

R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)
Qs = model.Qs = Var(model.S,model.k,within = NonNegativeReals)
Qw = model.Qw = Var(model.k,within = NonNegativeReals)

In [13]:
model.coste = Objective(expr = Qs['HPS',1]*80+Qs['LPS',3]*50 + Qw[5]*20)

In [14]:
nk = list(model.k)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-5, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
        R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
        )

model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
        R[s,k-1]+Qs[s,k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
        )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
        sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
        sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )

model.C41 = ConstraintList()
for k in nk:
        model.C41.add(
        sum(Q['LPS',j,k] for j in model.j)  == Qs['LPS',k]
        )

model.C42 = ConstraintList()
for k in nk:
        model.C42.add(
        sum(Q['HPS',j,k] for j in model.j)  == Qs['HPS',k]
        )

model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
        R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
        R[s,intmax]==0
        )

    
nii = list(model.k)[0:1]+list(model.k)[2:] 
model.C71 = ConstraintList()
for i in nii:
    model.C71.add(
        Qs['HPS',i]==0
    )

niii = list(model.k)[0:3]+list(model.k)[4:] 
model.C72 = ConstraintList()
for i in niii:
    model.C72.add(
        Qs['LPS',i]==0
    )

nn = list(model.k)[0:intmax]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
        Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['LPS','W',k] ==0
    )

model.C91 = ConstraintList()
for k in model.k:
    model.C91.add(
        Q['HPS','W',k] ==0
    )

model.C10 = ConstraintList()
for k in nk:
        model.C10.add(
        sum(Q[i,j,0] for i in model.i for j in model.j)  == 0
        )

In [15]:
results = SolverFactory('glpk').solve(model)
#model.pprint()
results.write()
Qh = value(model.Qs['HPS',1])
Ql = value(model.Qs['LPS',3])
Qw = value(model.Qw[5])
print('Cold utility = {0:2.2f}, LP Steam = {1:2.2f}, HP Steam = {2:2.2f}'.format(Qw, Ql, Qh))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 6550.0
  Upper bound: 6550.0
  Number of objectives: 1
  Number of constraints: 86
  Number of variables: 109
  Number of nonzeros: 241
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07532072067260742
# ----------------------------------------------------------
#   Solution Information
# ------------------------------

In [16]:
U = {'C1': [5, 60,200,190], 'C2': [5, 60,140,140], 'W':[0,60,75,75]}
U_df = pd.DataFrame(U, index=['LPS','HPS', 'H1', 'H2'])
U_df

,C1,C2,W
LPS,5,5,0
HPS,60,60,60
H1,200,140,75
H2,190,140,75


In [17]:
model.y=Var(model.s,model.w, within=Binary)

model.C11 = ConstraintList()
for s in model.s:
    for w in model.w:
        model.C11.add(
        sum(Q[s,w,k] for k in model.k)-U_df.loc[s,w]*model.y[s,w] <=0
        ) 

In [18]:
model.coste.deactivate()
model.sumy = Objective(expr=sum(model.y[s,w] for s in model.s for w in model.w))


results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 6.0
  Upper bound: 6.0
  Number of objectives: 1
  Number of constraints: 98
  Number of variables: 127
  Number of nonzeros: 324
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.0753183364868164
# ----------------------------------------------------------
#   Solution Information
# -------------------------------------

In [19]:
model.pprint()
optimal_values = {(i, j,k): value(model.Q[i,j,k]) for i in model.s for j in model.w for k in model.k}
df=pd.DataFrame.from_dict(optimal_values, orient="index", columns=['Q'])
df.to_excel('Datos/Resultados_Ejercicio5.xlsx', sheet_name="Results_Q")

26 Set Declarations
    C10_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C11_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C41_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any 